# Google Earth Engine Panel Data Creation

## Initialize

In [1]:
# !pip install geemap
#!pip install ee

In [2]:
# !pip install uszipcode

In [1]:
#GEE specific
import ee
import geemap
import math

#plotting and functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from time import time
from enum import Enum


In [2]:
#Initialize Google Earth Engine
#ee.Authenticate() #just needed the 1st time
ee.Initialize()

In [3]:
# Check if geemap is working as intended - plot the leaflet map
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## [OLD] Machine Learning Model

In [11]:
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))

In [4]:
# Define classes
CLASSES = ['water',
           'vegetation_trees',
           'vegetation_grass',
           'turf',
           'impervious',
           'soil']

OLD_CLASSES = ['water_pools'] + CLASSES[1:]

In [5]:
# Change classes to include lakes
NEW_CLASSES = OLD_CLASSES + ['water_natural']
NEW_CLASSES[0] = 'water_pools'

# Define the  bands
NBANDS = ['R', 
         'G', 
         'B', 
         'N', 
         'NDVI',
         'N_Entropy', 
         'N_Contrast', 
         'N_Gearys']

ALL_BANDS = NBANDS + ['R_Entropy',
                      'R_Contrast',
                      'R_Gearys',
                      'G_Entropy',
                      'G_Contrast',
                      'G_Gearys',
                      'B_Entropy',
                      'B_Contrast',
                      'B_Gearys']

# Select desired band set
BANDS = NBANDS


In [6]:
def apply_3bands(image, band):
    i_8_bit = image.select(band).toUint8()
    square = ee.Kernel.square(**{'radius': 4})
    entropy = i_8_bit.entropy(square)
    glcm = i_8_bit.glcmTexture(**{'size': 4})
    contrast = glcm.select(str(band)+'_contrast')
    
    # Create a list of weights for a 9x9 kernel.
    list = [1, 1, 1, 1, 1, 1, 1, 1, 1]
    # The center of the kernel is zero.
    centerList = [1, 1, 1, 1, 0, 1, 1, 1, 1]
    # Assemble a list of lists: the 9x9 kernel weights as a 2-D matrix.
    lists = [list, list, list, list, centerList, list, list, list, list]
    # Create the kernel from the weights.
    # Non-zero weights represent the spatial neighborhood.
    kernel = ee.Kernel.fixed(9, 9, lists, -4, -4, False)
    neighs = i_8_bit.neighborhoodToBands(kernel)
    gearys = i_8_bit.subtract(neighs).pow(2).reduce(ee.Reducer.sum()).divide(math.pow(9, 2))
    image = image.addBands(entropy.rename(str(band)+'_Entropy')).addBands(contrast.rename(str(band)+'_Contrast')).addBands(gearys.rename(str(band)+'_Gearys'))   
    return image

def add_neighborhood_bands(image):
    bands = ['R', 'G', 'B', 'N']
    for band in bands:
        image = apply_3bands(image, band)
    return image
    
def add_NDVI(image):
    image = image.addBands(image.normalizedDifference(['N','R']).rename('NDVI'))
    return image

def get_images(param_dict):
    source_image_collection = param_dict['source_image_collection']
    years = param_dict['years']
    counties = param_dict['counties']

    image_names = []
    images = []

    combos = list(itertools.product(years, counties.keys()))
    for i in combos:
        year = str(i[0])
        county = i[1]

        image_name = str(i[0])+'_'+i[1]
        image_names.append(image_name)

        image = ee.ImageCollection(source_image_collection)\
                                .filterDate(f'{year}-01-01', f'{year}-12-31')\
                                .select(['R','G','B','N'])\
                                .median().clip(counties[county])
        images.append(image)
        images_with_3band = list(map(add_neighborhood_bands, images))
        images_with_NDVI = list(map(add_NDVI, images_with_3band))
    return dict(zip(image_names, images_with_NDVI))

In [7]:
def get_TF_classified_image(image, bands, tf_model, classes, name='classification'):
    
    '''
    Use a TF model hosted on Google AI Platform to classify an EE image.
    '''
    
    # Select bands from training image for classification
    selected_image = image.select(bands)

    # Get the predictions
    predictions = tf_model.predictImage(selected_image.float().toArray())
    probabilities = predictions.arrayFlatten([classes])
    classified_image = predictions.arrayArgmax().arrayGet([0]).rename(name)
    
    return classified_image, predictions, probabilities

In [8]:
def get_ensembled_classified_image(image, bands, model_dicts):
    
    """
    Get the ensembled classified image by getting the max prediction probability
    for each pixel across all models. 
    
    Each dict in model_dicts should contain the following keys for a particular model:
        'model': the ee.Model object
        'classes': the classes that the model will try to predict
        'image_name': the name of the classified image from the model 
    """
    
    output_images = []
    combined_probs = None
    
    for model_dict in model_dicts:
        # Unpack the model metadata
        model = model_dict['model']
        classes = model_dict['classes']
        image_name = model_dict['image_name']
        
        # Predict on classified image
        temp_image, temp_preds, temp_probs = get_TF_classified_image(image, bands, model, classes, name=image_name)
        
        # If no combined_probs set, set it to temp_probs
        if combined_probs is None:
            combined_probs = temp_probs
        
        # Check if classes are not the same, then add missing bands 
        cur_prob_bands = set(combined_probs.bandNames().getInfo())
        add_cur_bands = list(cur_prob_bands - set(classes))
        add_new_bands = list(set(classes) - cur_prob_bands)
        
        if add_cur_bands:
            temp_probs = temp_probs.addBands(combined_probs, add_cur_bands)
        if add_new_bands:
            combined_probs = combined_probs.addBands(temp_probs, add_new_bands)
        
        # Get the max probs across the two images
        combined_probs = combined_probs.max(temp_probs)
        output_images.append(temp_image)
    
    # Get the final combined classification image based on maximum probabilities
    classified_image = combined_probs.toArray().arrayArgmax().arrayGet([0]).rename('combined_classification')
    output_images.append(classified_image)
    
    return output_images

In [12]:
training_image_params = {
        'source_image_collection' : 'USDA/NAIP/DOQQ',
        'years' : [2020],
        'counties': {'la_county': la_county}
         }

TRAINING_IMAGE = get_images(training_image_params)['2020_la_county']

In [13]:
Map = geemap.Map()

# Map.addLayer(TRAINING_IMAGE, {}, 'NAIP')

In [14]:
# Point to the TF model(s) to be used for inference
PROJECT = 'w210-351617'
VERSION_NAME = 'v0'
OLD_MODEL_NAME = 'CNN_Nbands_model'
NEW_MODEL_NAME = 'CNN_Nbands_sep_cls_wlake_model'

input_dim = [12,12]

# Point to the old model hosted on AI Platform.
old_tf_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=OLD_MODEL_NAME,
    version=VERSION_NAME,
    # Can be anything, but don't make it too big.
    inputTileSize=input_dim,
    # Note the names here need to match what was specified in the
    # output dictionary passed to the EEifier originally
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

# Point to the old model hosted on AI Platform.
new_tf_model = ee.Model.fromAiPlatformPredictor(
    projectName=PROJECT,
    modelName=NEW_MODEL_NAME,
    version=VERSION_NAME,
    # Can be anything, but don't make it too big.
    inputTileSize=input_dim,
    # Note the names here need to match what was specified in the
    # output dictionary passed to the EEifier originally
    outputBands={'output': {
        'type': ee.PixelType.float(),
        'dimensions': 1
      }
    },
)

model_dicts = [
    {'model': old_tf_model, 'classes': OLD_CLASSES, 'image_name':'old_classification'},
    {'model': new_tf_model, 'classes': NEW_CLASSES, 'image_name':'new_classification'},
]

# Classify the training image per model + ensembled
classified_images = get_ensembled_classified_image(TRAINING_IMAGE, BANDS, model_dicts)

In [15]:
classified_images

In [16]:
# Remap the combined image so lakes are classified as pools together
fromList = [0, 1, 2, 3, 4, 5, 6]
toList = [0, 1, 2, 3, 4, 5, 0]

classified_images[-1] = classified_images[-1].remap(fromList, toList)

old_legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911']
new_legend_colors = ['#0B6AEF', '#097407', '#0CE708', '#8C46D2' ,' #A1A8AF','#D47911', '#191970']


#water: #0B6AEF
#trees dark green #097407
#grass light green #0CE708
#turf purple #8C46D2
#impervious grey #A1A8AF
#soil: orange #D47911


colors_list = [old_legend_colors, new_legend_colors, old_legend_colors]

# for colors, image in zip(colors_list, classified_images):
#     layer_name = image.getInfo()['bands'][0]['id']
#     Map.addLayer(image, {'min': 0, 'max': len(colors)-1, 'palette': colors}, layer_name)

Map.addLayer(classified_images[0], {'min': 0, 'max': len(old_legend_colors)-1, 'palette': old_legend_colors}, 'CLASSIFIED')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [42]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…